# Commercial Bank Customer Retention Prediction

## APSTA-GE.2401: Statistical Consulting

## Scripts

Created on: 12/07/2020

Modified on: 12/08/2020

## Supervised Learning Models

----

### Description

This script contains the machine learning models.

### Research Design

The strategy of supervised learning is to train models using the `X_train` data and validate model performance using the `y_train` data. After training, we fit the model to the `X_test` data. The model will then generate predictions, `y_test`, based on `X_test`. 

To increase model performance, we splited the train set into two sets: 80% of the train data goes to the `X_train` set and 20% of the data goes to the `X_test` set. Then, we conducted a 5-fold cross validation and selected the best performed model output. We also find tuned hyperparameters using randomized search.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler

print('SUCCESS! All modules are imported.')

SUCCESS! All modules are imported.


----

In [3]:
X = pd.read_csv('../data/preprocess/X_train.csv')
y = pd.read_csv('../data/preprocess/y_train.csv')
X_true = pd.read_csv('../data/preprocess/X_test.csv')

### Dummy Coding

In [5]:
X.head().transpose()

,0,1,2,3,4
cust_no,0xb2d640a6,0xb2d4dd44,0xb2d69fef,0xb2d42c89,0xb2d90b58
X1,51000,0,20000,0,0
X2,0,0,0,0,0
X3,256.54,208.51,3265.03,1.41,13.38
X4,0,0,0,0,0
X5,0,0,0,333.01,0
X6,0,0,0,0,0
X7,0,0,0,0,0
X8,0,0,200000,0,0
B1,1,0,0,23,0
